# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [2]:
using Pkg
Pkg.add("DecisionTree")
Pkg.add("ScikitLearn")

   Updating registry at `C:\Users\User\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed DecisionTree ─ v0.10.1
   Updating `C:\Users\User\.julia\environments\v1.4\Project.toml`
  [7806a523] + DecisionTree v0.10.1
   Updating `C:\Users\User\.julia\environments\v1.4\Manifest.toml`
  [7806a523] + DecisionTree v0.10.1
  Resolving package versions...
  Installed ProgressMeter ─────── v1.2.0
  Installed PyPlot ────────────── v2.9.0
  Installed ElasticArrays ─────── v1.2.0
  Installed ArrayLayouts ──────── v0.2.6
  Installed JLD2 ──────────────── v0.1.13
  Installed IRTools ───────────── v0.3.2
  Installed Documenter ────────── v0.24.11
  Installed Adapt ─────────────── v1.0.1
  Installed ZygoteRules ───────── v0.2.0
  Installed ElasticPDMats ─────── v0.2.1
  Installed NBInclude ─────────── v2.2.0
  Installed FastGaussQuadrature ─ v0.4.2
  Installed NNlib ─────────────── v0.6.6
  Installed Zygote ────────────── v0.4.20
  Installed GaussianMixtures ──── v0.3.1
  Installed ScikitLearn ───────── v0.5.1
  Installed GaussianProcesses ─── v0

In [63]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics
using ScikitLearn.CrossValidation: cross_val_score

## 讀取資料

In [7]:
biopsy = dataset("MASS", "biopsy")
describe(biopsy)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,Type
1,ID,,1000025,,95719,645,,String
2,V1,4.41774,1,4.0,10,,,Int32
3,V2,3.13448,1,1.0,10,,,Int32
4,V3,3.20744,1,1.0,10,,,Int32
5,V4,2.80687,1,1.0,10,,,Int32
6,V5,3.21602,1,2.0,10,,,Int32
7,V6,3.54466,1,1.0,10,,16,"Union{Missing, Int32}"
8,V7,3.43777,1,3.0,10,,,Int32
9,V8,2.86695,1,1.0,10,,,Int32


In [42]:
dropmissing!(biopsy)

indecies = MLDataUtils.shuffleobs(collect(1:nrow(biopsy)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);
train = biopsy[train_ind, :]
test = biopsy[test_ind, :]

features_train = Int32.(Matrix(train[:,2:10]))
label_train = string.(train[!,:Class])
features_test = Int32.(Matrix(test[:,2:10]))
label_test = string.(test[!,:Class])

137-element Array{String,1}:
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"
 "benign"
 "malignant"
 ⋮
 "malignant"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [45]:
model = DecisionTree.RandomForestClassifier(n_trees=50, max_depth=2)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [46]:
DecisionTree.fit!(model, features_train, label_train)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             ["benign", "malignant"]
ensemble:            Ensemble of Decision Trees
Trees:      50
Avg Leaves: 3.98
Avg Depth:  2.0

In [56]:
size(features_test)[1]
features_test[1,:]
DecisionTree.predict(model, features_test[1,:])

"benign"

In [60]:
mPassCnt = 0
mTotalCnt = size(features_test)[1]
for idx in 1:size(features_test)[1]
    ans = DecisionTree.predict(model, features_test[idx,:])
#     println(ans)
    if ans==label_test[idx]
        mPassCnt+=1
    end
end

println("Pass Count = ", mPassCnt)
println("Pass Rate = ", mPassCnt/mTotalCnt*100, " %")

Pass Count = 133
Pass Rate = 97.08029197080292 %


In [64]:
accuracy = cross_val_score(model, features_train, label_train, cv=5)

5-element Array{Float64,1}:
 0.9454545454545454
 1.0
 0.963302752293578
 0.981651376146789
 0.963302752293578

In [65]:
accuracy = cross_val_score(model, features_test, label_test, cv=5)

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 0.9629629629629629
 0.9615384615384616